# Project Step 2: Training & Tracking (Hydra + W&B)

**Objective**: Train a YOLOv8 model while managing configs with **Hydra** and tracking results with **Weights & Biases**.

**Why?**
- **Hydra**: Avoids hardcoding parameters (`epochs=50`). Allows easy overrides.
- **W&B**: Visualizes loss curves and validates if the model is learning.

---

## 📚 Steps
1. **Setup Config**: Create `conf/train.yaml`.
2. **Load Config**: Use Hydra's Compose API.
3. **Train Model**: Run YOLOv8 training.
4. **Analyze Results**: View metrics in W&B.

## 1. Setup Config (Hydra)

We will programmatically create a YAML config file in `../conf/train.yaml`.
In a real project, you would edit this file manually.

In [ ]:
import os
import yaml
from hydra import compose, initialize
from omegaconf import OmegaConf

CONF_DIR = "../conf"
os.makedirs(CONF_DIR, exist_ok=True)

# Define the config structure
config_data = """
defaults:
  - _self_

experiment:
  name: "yolo_baseline"
  project: "cv_mlops_project"

model:
  name: "yolov8n.pt"  # Nano model (fastest)

training:
  epochs: 5
  batch_size: 16
  img_size: 640
  device: "cpu" # or 0 for GPU
  workers: 2
"""

with open(os.path.join(CONF_DIR, "train.yaml"), "w") as f:
    f.write(config_data)

print("✅ Config created at ../conf/train.yaml")

## 2. Load Config

We simulate running `python train.py` by using Hydra's `initialize` and `compose`.

In [ ]:
# Initialize Hydra (context manager needed for Jupyter)
with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="train")
    
print("--- Loaded Config ---")
print(OmegaConf.to_yaml(cfg))

## 3. Train Model (YOLOv8)

We use the `ultralytics` library. It has built-in support for W&B.
Just calling `model.train()` with `project=` argument might trigger it, but let's be explicit.

In [ ]:
from ultralytics import YOLO
import wandb

# 1. Initialize W&B (Optional: Ultralytics does this automatically if installed, 
# but manual init ensures we control the project name)
wandb.init(
    project=cfg.experiment.project,
    name=cfg.experiment.name,
    config=OmegaConf.to_container(cfg)
)

# 2. Load Model
model = YOLO(cfg.model.name)

# 3. Train
# Note: coco128.yaml is built-in to Ultralytics. 
# If using custom data, point to absolute path of data.yaml.
print(f"Starting training for {cfg.training.epochs} epochs...")

results = model.train(
    data="coco128.yaml",
    epochs=cfg.training.epochs,
    batch=cfg.training.batch_size,
    imgsz=cfg.training.img_size,
    device=cfg.training.device,
    workers=cfg.training.workers,
    project="../models",
    name=cfg.experiment.name,
    exist_ok=True # Overwrite existing exp folder
)

wandb.finish()

## 4. Analyze Results

The training artifacts are saved in `../models/yolo_baseline`.
We can inspect the validation metrics.

In [ ]:
import pandas as pd

# Ultralytics saves a results.csv
results_path = f"../models/{cfg.experiment.name}/results.csv"

if os.path.exists(results_path):
    df = pd.read_csv(results_path)
    # Clean column names (strip whitespace)
    df.columns = df.columns.str.strip()
    
    # Plot mAP50-95
    print("--- Training Metrics (Last 5 Epochs) ---")
    print(df[['epoch', 'train/box_loss', 'metrics/mAP50-95(B)']].tail())
    
    # Simple Plot
    df.plot(x='epoch', y='metrics/mAP50-95(B)', title="mAP over Epochs")
else:
    print("Training failed or results file not found.")

## Next Step

You have a trained model! But it's in PyTorch (`.pt`) format. For production, we need something faster.

Proceed to `03_optimization_onnx.ipynb`.